In [20]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import monotonically_increasing_id,col
from pyspark.sql.types import *

In [ ]:
#Define entrypoint Spark_App

In [2]:
conf=SparkConf()
conf.setAppName("spark_iceberg_lession")
conf.setMaster("local[6]")
conf.set("spark.sql.extensions","org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
conf.set("spark.driver.memory","4G")
conf.set("spark.executors.memory","2G")
conf.set("spark.executors.cores",2)


In [3]:
spark=SparkSession.builder.config(conf=conf).getOrCreate()
spark

23/05/11 15:06:13 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [4]:
%%sql
CREATE DATABASE IF NOT EXISTS acb_test

23/05/11 15:06:18 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


++
||
++
++

In [5]:
spark.sql("CREATE TABLE IF NOT EXISTS demo.acb_test.PIT " +
    "(uuid string not null, key string, current_flag int,sysdate long) using iceberg PARTITIONED BY(sysdate)")

DataFrame[]

In [7]:
%%sql
use acb_test

++
||
++
++

In [8]:
%%sql
show tables 

namespace,tableName,isTemporary
acb_test,PIT,False


In [9]:
%%sql
select * from PIT

uuid,key,current_flag,sysdate


In [ ]:
# inital load data to PIT

In [10]:
spark.sql("INSERT INTO demo.acb_test.PIT " +
    "VALUES ('1', 'A', 1,20230506),('2', 'B', 1,20230507)")

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


DataFrame[]

In [11]:
%%sql
select * from PIT

uuid,key,current_flag,sysdate
1,A,1,20230506
2,B,1,20230507


In [12]:
%%sql
describe extended PIT

col_name,data_type,comment
uuid,string,
key,string,
current_flag,int,
sysdate,bigint,
,,
# Partitioning,,
Part 0,sysdate,
,,
# Metadata Columns,,
_spec_id,int,


In [ ]:
# new data generators

In [14]:
spark.sql("select '4' as uuid, 'B' as key,1 as current_flag, 20230508 as sysdate union select '5' as uuid, 'C' as key,1 as current_flag, 20230516 as sysdate").createOrReplaceTempView("new_data")

In [15]:
%%sql
select * from new_data

uuid,key,current_flag,sysdate
4,B,1,20230508
5,C,1,20230516


In [24]:
merge_table='demo.acb_test.PIT'
new_data_df='new_data'

In [32]:
update_curent_flag=f" MERGE INTO {merge_table} old USING (select DISTINCT * from {new_data_df}) new on old.key=new.key WHEN MATCHED AND new.sysdate>old.sysdate THEN  UPDATE SET old.current_flag=0 WHEN NOT MATCHED THEN INSERT * "

In [33]:
spark.sql(update_curent_flag)

DataFrame[]

In [34]:
%%sql
select * from PIT

uuid,key,current_flag,sysdate
1,A,1,20230506
2,B,0,20230507
5,C,1,20230516


In [38]:
add_latest_record=f"MERGE INTO {merge_table} old USING (select DISTINCT * from new_data) new on old.key=new.key AND new.sysdate<=old.sysdate WHEN NOT MATCHED THEN INSERT *"

In [39]:
spark.sql(add_latest_record)

DataFrame[]

In [40]:
%%sql
select * from PIT

uuid,key,current_flag,sysdate
2,B,0,20230507
4,B,1,20230508
5,C,1,20230516
1,A,1,20230506
